# Train Astro R-CNN

Train Mark R-CNN on PhoSim data

In [4]:
import numpy as np
import matplotlib

import train
from train import PhoSimDataset

# Root directory of the project
ROOT_DIR = os.path.abspath("./Mask_RCNN")
#OUT_DIR = os.path.abspath("./phosim_release/output/")
OUT_DIR = os.path.abspath("./trainingset/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline


ModuleNotFoundError: No module named 'keras'

In [5]:
## Config

config = DESConfig()
config.display()

NameError: name 'DESConfig' is not defined

In [3]:
## DATASET

# Training dataset
dataset_train = PhoSimDataset()
dataset_train.load_sources()
dataset_train.prepare()

# Validation dataset
dataset_val = PhoSimDataset()
dataset_val.load_sources()
dataset_val.prepare()

NameError: name 'PhoSimDataset' is not defined

In [6]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 1)
for image_id in image_ids:
    print("loading image to visualize. Be patient...")
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## CREATE MODEL

# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

# Load weights trained on MS COCO, but skip layers that
# are different due to the different number of classes
# See README for instructions to download the COCO weights
model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])


NameError: name 'dataset_train' is not defined

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=1,
            layers='heads')

# Fine tune all layers
# Passing layers="all" trains all layers. You can also
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2,
            layers="all")

# Save weights
model_path = os.path.join(MODEL_DIR, "astro_rcnn_des.h5")
model.keras_model.save_weights(model_path)
